## Automated Sales Outreach Agent

Welcome to your Automated Sales Outreach project!

We're going to build a comprehensive Agent system for generating and sending cold sales outreach emails using Gemini and Brevo:
1. **Agent workflow**: Parallel generation of email drafts.
2. **Selection**: An agent to pick the best draft.
3. **Tools**: Integrating Brevo (Sendinblue) to actually send emails.
4. **Handoffs**: Specialist agents for subject lines and HTML formatting.

## Before we start - Setup:

Ensure you have a **Brevo** account (formerly Sendinblue) at: https://www.brevo.com/

1.  **API Key**: Go to your Profile > SMTP & API > Generate a new API Key. Add this to your `.env` file as `BREVO_API_KEY`.
2.  **Sender Verification**: Ensure you have a verified sender email in Brevo.
3.  **Gemini API Key**: Ensure you have `GEMINI_API_KEY` (or `GOOGLE_API_KEY`) in your `.env` for the AI model.

We will load these environments variables below.

In [48]:
import os
import asyncio
import sib_api_v3_sdk
from sib_api_v3_sdk.rest import ApiException
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict


# 1. Force reload environment variables
load_dotenv(override=True)

# 2. Verify Keys exist before running
if not os.getenv("BREVO_API_KEY"):
    raise ValueError("Missing BREVO_API_KEY in .env file")

# Handle GOOGLE_API_KEY / GEMINI_API_KEY mismatch fallback
if not os.getenv("GEMINI_API_KEY") and os.getenv("GOOGLE_API_KEY"):
    os.environ["GEMINI_API_KEY"] = os.environ["GOOGLE_API_KEY"]

if not os.getenv("GEMINI_API_KEY"):
    raise ValueError("Missing GEMINI_API_KEY (or GOOGLE_API_KEY) in .env file")

# The API key will be automatically picked up by LiteLLM / Agent SDK
OPENROUTER_API_KEY = os.environ.get("OPENROUTER_API_KEY")
if not OPENROUTER_API_KEY:
    raise ValueError("OPENROUTER_API_KEY not found in environment variables")

In [49]:
# --- Configuration ---
# UPDATE THIS to your verified Brevo sender
SENDER_EMAIL = {"email": "abelmarie49@gmail.com", "name": "Abel Marie"} 
RECIPIENT_EMAIL = [{"email": "birukabere4@gmail.com", "name": "Dick Head"}]


In [50]:
# FREE MODELS (AUTO FALLBACK)
# -------------------------------------------------
FREE_MODELS = [
    "litellm/openrouter/meta-llama/llama-3.2-3b-instruct:free",
    "litellm/openrouter/mistralai/mistral-7b-instruct:free",
    "litellm/openrouter/tngtech/deepseek-r1t2-chimera:free",
    "litellm/openrouter/meta-llama/llama-3.3-70b-instruct:free",
    "litellm/openrouter/nousresearch/hermes-3-llama-3.1-405b:free",
]


In [51]:
# --- Helper: Brevo Configuration ---
def get_brevo_api_instance():
    configuration = sib_api_v3_sdk.Configuration()
    configuration.api_key['api-key'] = os.environ.get('BREVO_API_KEY')
    return sib_api_v3_sdk.TransactionalEmailsApi(sib_api_v3_sdk.ApiClient(configuration))

## Step 1: Agent Workflow

We will define three distinct personalities for our sales agents to generate different styles of copy.

In [52]:
instructions1 = "You are a sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails."

instructions2 = "You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

instructions3 = "You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."

In [53]:
AGENT_DEFINITIONS = [
    ("Professional Sales Agent", instructions1),
    ("Engaging Sales Agent", instructions2),
    ("Busy Sales Agent", instructions3),
]


In [54]:
prompt = "Write a short cold sales email"

# Sequental Generation

In [ ]:
async def run_agents():
    for agent_name, instructions in AGENT_DEFINITIONS:
        print(f"\n--- {agent_name} ---\n")

        last_error = None

        for model in FREE_MODELS:
            print(f"🔁 Trying model → {model}")

            agent = Agent(
                name=agent_name,
                instructions=instructions,
                model=model
            )

            try:
                result = Runner.run_streamed(agent, input=prompt)

                async for event in result.stream_events():
                    if event.type == "raw_response_event":
                        if hasattr(event.data, "delta") and event.data.delta:
                            print(event.data.delta, end="", flush=True)

                print("\n✅ Success\n")
                break

            except Exception as e:
                last_error = e
                print(f"⚠️ Model failed: {model}")
                print(f"Reason: {e}\n")
                await asyncio.sleep(1)

        else:
            print("❌ All free models failed.")
            raise last_error


In [56]:
# In Jupyter, you can await directly in the cell
await run_agents()



--- Professional Sales Agent ---

🔁 Trying model → litellm/openrouter/meta-llama/llama-3.2-3b-instruct:free

Provider List: https://docs.litellm.ai/docs/providers


Provider List: https://docs.litellm.ai/docs/providers

Subject: Simplify Your SOC2 Comp

OPENAI_API_KEY is not set, skipping trace export


liance with AI-Powered Solutions

Dear [Recipient's Name],

As a compliance and risk management professional, you understand the importance of maintaining SOC2 compliance. However, navigating the complex auditing process can be time-consuming and costly.

At ComplAI, we offer a cutting-edge SaaS solution that empowers organizations like yours to ensure SOC2 compliance with ease. Our AI-powered tool streamlines the audit preparation process, providing real-time reporting and automated risk assessments.

By partnering with ComplAI, you can:

- Improve audit readiness with our comprehensive audit preparations
- Enhance stakeholder confidence through transparent reporting
- Reduce audit costs by up to 30%

Discover how ComplAI can simplify your SOC2 compliance journey. Schedule a personalized demo today and experience the future of audit preparedness.

Best regards,
[Your Name]
ComplAI Sales Team
Provider List: https://docs.litellm.ai/docs/providers


Provider List: https://docs.litellm.ai

OPENAI_API_KEY is not set, skipping trace export


Here is a short cold sales email:

Subject: Simplify Your Compliance Checks with ComplAI

Hi [Name],

As the year winds down, many of us are reflecting on the challenges we faced in meeting our compliance goals. Ensuring SOC2 compliance can be a nightmare, but it doesn't have to be.

Our team at ComplAI has worked with numerous companies like yours to streamline their compliance processes. With our cutting-edge SaaS tool, you can identify potential risks, stay on top of your compliance posture, and pass audits with flying colors.

We'd love to show you how our platform can save you time and headaches in the new year. Would you be open to a quick demo to see how ComplAI can help you simplify your compliance checks?

Best,
[Your
 NameProvider List: https://docs.litellm.ai/docs/providers

]
ComplAI Team
Provider List: https://docs.litellm.ai/docs/providers


✅ Success


--- Busy Sales Agent ---

🔁 Trying model → litellm/openrouter/meta-llama/llama-3.2-3b-instruct:free

Provider List: http

OPENAI_API_KEY is not set, skipping trace export


🔁 Trying model → litellm/openrouter/mistralai/mistral-7b-instruct:free

Provider List: https://docs.litellm.ai/docs/providers


Provider List: https://docs.litellm.ai/docs/providers

 <s> 
Provider List: https://docs.litellm.ai/docs/providers


Provider List: https://docs.litellm.ai/docs/providers


✅ Success



### Parallel Generation

Now we generate drafts from all three agents in parallel.

In [58]:
async def run_agent_with_fallback(agent_name, instructions, message):
    last_error = None

    for model in FREE_MODELS:
        print(f"🔁 {agent_name} → Trying model: {model}")

        agent = Agent(
            name=agent_name,
            instructions=instructions,
            model=model,
        )

        try:
            result = await Runner.run(agent, message)
            print(f"✅ {agent_name} succeeded with {model}\n")
            return result.final_output

        except Exception as e:
            last_error = e
            print(f"⚠️ {agent_name} failed on {model}")
            print(f"Reason: {e}\n")
            await asyncio.sleep(0.5)

    print(f"❌ {agent_name}: all free models failed.")
    raise last_error


In [59]:
message = "Write a cold sales email"

with trace("Parallel cold emails"):
    outputs = await asyncio.gather(
        *[
            run_agent_with_fallback(agent_name, instructions, message)
            for agent_name, instructions in AGENT_DEFINITIONS
        ]
    )


🔁 Professional Sales Agent → Trying model: litellm/openrouter/meta-llama/llama-3.2-3b-instruct:free
🔁 Engaging Sales Agent → Trying model: litellm/openrouter/meta-llama/llama-3.2-3b-instruct:free
🔁 Busy Sales Agent → Trying model: litellm/openrouter/meta-llama/llama-3.2-3b-instruct:free

Provider List: https://docs.litellm.ai/docs/providers


Provider List: https://docs.litellm.ai/docs/providers


Provider List: https://docs.litellm.ai/docs/providers


Provider List: https://docs.litellm.ai/docs/providers


Provider List: https://docs.litellm.ai/docs/providers


Provider List: https://docs.litellm.ai/docs/providers


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/n

OPENAI_API_KEY is not set, skipping trace export


🔁 Professional Sales Agent → Trying model: litellm/openrouter/mistralai/mistral-7b-instruct:free
🔁 Busy Sales Agent → Trying model: litellm/openrouter/mistralai/mistral-7b-instruct:free
🔁 Engaging Sales Agent → Trying model: litellm/openrouter/mistralai/mistral-7b-instruct:free

Provider List: https://docs.litellm.ai/docs/providers


Provider List: https://docs.litellm.ai/docs/providers


Provider List: https://docs.litellm.ai/docs/providers


Provider List: https://docs.litellm.ai/docs/providers


Provider List: https://docs.litellm.ai/docs/providers


Provider List: https://docs.litellm.ai/docs/providers


Provider List: https://docs.litellm.ai/docs/providers


Provider List: https://docs.litellm.ai/docs/providers


Provider List: https://docs.litellm.ai/docs/providers


Provider List: https://docs.litellm.ai/docs/providers

✅ Professional Sales Agent succeeded with litellm/openrouter/mistralai/mistral-7b-instruct:free


Provider List: https://docs.litellm.ai/docs/providers


Provide

In [60]:
for output in outputs:
    print(output + "\n\n")


 to a potential customer who is preparing for a SOC2 audit and is a mid-sized SaaS company. The customer is a CTO who is preparing for their first SOC2 audit.

---

**Subject:** Simplify Your First SOC2 Audit with ComplAI

Hi [First Name],

I hope this email finds you well. As a CTO preparing for your first SOC2 audit, I understand the complexity and time investment required to ensure compliance. At ComplAI, we specialize in helping mid-sized SaaS companies like yours streamline the SOC2 audit process with our AI-powered SaaS tool.

Our platform automates evidence collection, policy generation, and risk assessments, significantly reducing the manual effort and stress associated with SOC2 compliance. With ComplAI, you can focus on your core business while we handle the compliance heavy lifting.

Would you be open to a quick 15-minute call next week to discuss how ComplAI can support your audit preparation? I’d love to learn more about your current process and share how we’ve helped simi

### Selection Agent

We need a "Sales Picker" agent to review the drafts and choose the best one.

In [61]:
sales_picker_instructions = (
    "You pick the best cold sales email from the given options. "
    "Imagine you are a customer and pick the one you are most likely to respond to. "
    "Do not give an explanation; reply with the selected email only."
)


In [62]:
async def run_with_fallback(agent_name, instructions, message):
    last_error = None

    for model in FREE_MODELS:
        print(f"🔁 {agent_name} → Trying model: {model}")

        agent = Agent(
            name=agent_name,
            instructions=instructions,
            model=model,
        )

        try:
            result = await Runner.run(agent, message)
            print(f"✅ {agent_name} succeeded with {model}\n")
            return result.final_output

        except Exception as e:
            last_error = e
            print(f"⚠️ {agent_name} failed on {model}")
            print(f"Reason: {e}\n")
            await asyncio.sleep(0.5)

    print(f"❌ {agent_name}: all free models failed.")
    raise last_error


In [63]:
message = "Write a cold sales email"

with trace("Selection from sales people"):
    outputs = await asyncio.gather(
        run_with_fallback("Professional Sales Agent", instructions1, message),
        run_with_fallback("Engaging Sales Agent", instructions2, message),
        run_with_fallback("Busy Sales Agent", instructions3, message),
    )


🔁 Professional Sales Agent → Trying model: litellm/openrouter/meta-llama/llama-3.2-3b-instruct:free
🔁 Engaging Sales Agent → Trying model: litellm/openrouter/meta-llama/llama-3.2-3b-instruct:free
🔁 Busy Sales Agent → Trying model: litellm/openrouter/meta-llama/llama-3.2-3b-instruct:free

Provider List: https://docs.litellm.ai/docs/providers


Provider List: https://docs.litellm.ai/docs/providers


Provider List: https://docs.litellm.ai/docs/providers


Provider List: https://docs.litellm.ai/docs/providers


Provider List: https://docs.litellm.ai/docs/providers


Provider List: https://docs.litellm.ai/docs/providers



OPENAI_API_KEY is not set, skipping trace export



Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.

⚠️ Engaging Sales Agent failed on litellm/openrouter/meta-llama/llama-3.2-3b-instruct:free
Reason: litellm.RateLimitError: RateLimitError: OpenrouterException - {"error":{"message":"Provider returned error","code":429,"metadata":{"raw":"meta-llama/llama-3.2-3b-instruct:free is temporarily rate-limited upstream. Please retry shortly, or add your own key to accumulate your rate limits: https://openrouter.ai/settings/integrations","provider_name":"Venice"}},"user_id":"user_35bH96ZYgxpOm3JCUhaUt3Iq1s8"}


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.

⚠️ Busy Sales Agent failed on litel

OPENAI_API_KEY is not set, skipping trace export



Provider List: https://docs.litellm.ai/docs/providers


Provider List: https://docs.litellm.ai/docs/providers


Provider List: https://docs.litellm.ai/docs/providers


Provider List: https://docs.litellm.ai/docs/providers

✅ Busy Sales Agent succeeded with litellm/openrouter/mistralai/mistral-7b-instruct:free



OPENAI_API_KEY is not set, skipping trace export



Provider List: https://docs.litellm.ai/docs/providers


Provider List: https://docs.litellm.ai/docs/providers


Provider List: https://docs.litellm.ai/docs/providers


Provider List: https://docs.litellm.ai/docs/providers

✅ Engaging Sales Agent succeeded with litellm/openrouter/mistralai/mistral-7b-instruct:free


Provider List: https://docs.litellm.ai/docs/providers


Provider List: https://docs.litellm.ai/docs/providers


Provider List: https://docs.litellm.ai/docs/providers


Provider List: https://docs.litellm.ai/docs/providers

✅ Professional Sales Agent succeeded with litellm/openrouter/mistralai/mistral-7b-instruct:free



In [64]:
emails = "Cold sales emails:\n\n" + "\n\nEmail:\n\n".join(outputs)


In [65]:
best_email = await run_with_fallback(
    agent_name="Sales Picker",
    instructions=sales_picker_instructions,
    message=emails,
)


🔁 Sales Picker → Trying model: litellm/openrouter/meta-llama/llama-3.2-3b-instruct:free

Provider List: https://docs.litellm.ai/docs/providers


Provider List: https://docs.litellm.ai/docs/providers



OPENAI_API_KEY is not set, skipping trace export



Provider List: https://docs.litellm.ai/docs/providers


Provider List: https://docs.litellm.ai/docs/providers


Provider List: https://docs.litellm.ai/docs/providers


Provider List: https://docs.litellm.ai/docs/providers

✅ Sales Picker succeeded with litellm/openrouter/meta-llama/llama-3.2-3b-instruct:free



In [66]:
print(f"Best sales email:\n{best_email}")


Best sales email:
Subject: Streamline Your SOC2 Compliance with AI-Powered Efficiency


## Step 2: Use of Tools

Now we convert our functionality into Tools so the agents can take actions.

1. **Email Sending Tool**: Wraps the Brevo API.
2. **Agent Tools**: Wraps our personas so a manager can call them.

In [68]:
@function_tool
def send_email(body: str):
    """ Send out an email with the given body to all sales prospects """
    print(f"--- TOOL CALLED: Sending Email via Brevo ---")
    api_instance = get_brevo_api_instance()
    send_smtp_email = sib_api_v3_sdk.SendSmtpEmail(
        to=RECIPIENT_EMAIL,
        sender=SENDER_EMAIL,
        subject="Sales email",
        text_content=body
    )
    try:
        api_instance.send_transac_email(send_smtp_email)
        return {"status": "success"}
    except ApiException as e:
        return {"status": "error", "message": str(e)}


In [69]:
tool1 = sales_agent1.as_tool(
    tool_name="sales_agent1",
    tool_description="Write a cold sales email"
)

tool2 = sales_agent2.as_tool(
    tool_name="sales_agent2",
    tool_description="Write a cold sales email"
)

tool3 = sales_agent3.as_tool(
    tool_name="sales_agent3",
    tool_description="Write a cold sales email"
)

tools = [tool1, tool2, tool3, send_email]


### Sales Manager Agent

The Sales Manager coordinates the drafting and sending process.

In [70]:
manager_instructions = """
You are a Sales Manager.
1. Use sales_agent1, sales_agent2, and sales_agent3 to generate 3 drafts.
2. Pick the best one.
3. Use send_email to send ONLY the best one.
"""


In [71]:
async def run_manager_with_fallback(message):
    last_error = None

    for model in FREE_MODELS:
        print(f"🔁 Sales Manager → Trying model: {model}")

        sales_manager = Agent(
            name="Sales Manager",
            instructions=manager_instructions,
            tools=tools,
            model=model,
        )

        try:
            result = await Runner.run(sales_manager, message)
            print(f"✅ Sales Manager succeeded with {model}\n")
            return result.final_output

        except Exception as e:
            last_error = e
            print(f"⚠️ Sales Manager failed on {model}")
            print(f"Reason: {e}\n")
            await asyncio.sleep(0.5)

    print("❌ Sales Manager: all free models failed.")
    raise last_error


In [72]:
print("\n--- 3. Running Sales Manager (Tool Use) ---")

await run_manager_with_fallback(
    "Send a cold sales email to the CEO"
)



--- 3. Running Sales Manager (Tool Use) ---
🔁 Sales Manager → Trying model: litellm/openrouter/meta-llama/llama-3.2-3b-instruct:free

Provider List: https://docs.litellm.ai/docs/providers


Provider List: https://docs.litellm.ai/docs/providers



OPENAI_API_KEY is not set, skipping trace export



Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.

⚠️ Sales Manager failed on litellm/openrouter/meta-llama/llama-3.2-3b-instruct:free
Reason: litellm.NotFoundError: NotFoundError: OpenrouterException - {"error":{"message":"No endpoints found that support tool use. To learn more about provider routing, visit: https://openrouter.ai/docs/guides/routing/provider-selection","code":404}}

🔁 Sales Manager → Trying model: litellm/openrouter/mistralai/mistral-7b-instruct:free

Provider List: https://docs.litellm.ai/docs/providers


Provider List: https://docs.litellm.ai/docs/providers



OPENAI_API_KEY is not set, skipping trace export



Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.

⚠️ Sales Manager failed on litellm/openrouter/mistralai/mistral-7b-instruct:free
Reason: litellm.Timeout: Timeout Error: OpenrouterException - litellm.Timeout: Connection timed out. Timeout passed=600.0, time taken=600.783 seconds

🔁 Sales Manager → Trying model: litellm/openrouter/tngtech/deepseek-r1t2-chimera:free

Provider List: https://docs.litellm.ai/docs/providers


Provider List: https://docs.litellm.ai/docs/providers


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.

⚠️ Sales Manager failed on litellm/openrouter/tngtech/deepseek-r1t2-chimera:free
Reason: litellm.NotFoundError: NotFoundError: OpenrouterException - {"error":{"message":"No endpoints found that support tool use. To learn more about provider routing, visit: https:/

OPENAI_API_KEY is not set, skipping trace export



Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.

⚠️ Sales Manager failed on litellm/openrouter/meta-llama/llama-3.3-70b-instruct:free
Reason: litellm.BadRequestError: OpenrouterException - {"error":{"message":"Provider returned error","code":400,"metadata":{"raw":"{\"detail\":\"Tools are not supported in streaming mode.\"}","provider_name":"ModelRun"}},"user_id":"user_35bH96ZYgxpOm3JCUhaUt3Iq1s8"}

🔁 Sales Manager → Trying model: litellm/openrouter/nousresearch/hermes-3-llama-3.1-405b:free

Provider List: https://docs.litellm.ai/docs/providers


Provider List: https://docs.litellm.ai/docs/providers


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.

⚠️ Sales Manager failed on litellm/openrouter/nousresearch/hermes-3-llama-3.1-405b:free
Reason: litellm.NotFoundError: NotFoundError: Op

Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x00000217DBFEE600>
C:\Users\Y\AppData\Roaming\uv\python\cpython-3.12.12-windows-x86_64-none\Lib\ast.py:280: RuntimeWarning: coroutine 'run_agents' was never awaited
  yield item


NotFoundError: litellm.NotFoundError: NotFoundError: OpenrouterException - {"error":{"message":"No endpoints found that support tool use. To learn more about provider routing, visit: https://openrouter.ai/docs/guides/routing/provider-selection","code":404}}

## Step 3: Agent Handoffs

For more complex workflows, we use **Handoffs**. Here, we'll create a specialized `Email Manager` that handles formatting (HTML) and Subject lines before sending.

In [73]:
# Define Specialist Agents
subject_writer = Agent(
    name="Subject Writer",
    instructions="Write a catchy subject line."
)

subject_tool = subject_writer.as_tool(
    tool_name="subject_writer",
    tool_description="Write a subject"
)

html_converter = Agent(
    name="HTML Converter",
    instructions="Convert text to HTML."
)

html_tool = html_converter.as_tool(
    tool_name="html_converter",
    tool_description="Convert to HTML"
)


In [74]:
@function_tool
def send_html_email(subject: str, html_body: str):
    """ Send HTML email """
    print(f"--- TOOL CALLED: Sending HTML Email via Brevo ---")
    api_instance = get_brevo_api_instance()
    send_smtp_email = sib_api_v3_sdk.SendSmtpEmail(
        to=RECIPIENT_EMAIL,
        sender=SENDER_EMAIL,
        subject=subject,
        html_content=html_body
    )
    try:
        api_instance.send_transac_email(send_smtp_email)
        return {"status": "success"}
    except ApiException as e:
        return {"status": "error", "message": str(e)}


In [75]:
emailer_agent_instructions = "Use tools to write subject, convert to HTML, then send."

# Tools: subject, html, send
tools_for_emailer = [subject_tool, html_tool, send_html_email]


In [76]:
final_manager_instructions = (
    "Generate 3 drafts using tools, pick the best, then handoff to Email Manager."
)

# Drafting tools: sales agents
drafting_tools = [tool1, tool2, tool3]


In [77]:
async def run_manager_fallback(agent_name, instructions, message, tools=None, handoffs=None):
    last_error = None

    for model in FREE_MODELS:
        print(f"🔁 {agent_name} → Trying model: {model}")

        agent = Agent(
            name=agent_name,
            instructions=instructions,
            tools=tools,
            handoffs=handoffs,
            model=model,
        )

        try:
            result = await Runner.run(agent, message)
            print(f"✅ {agent_name} succeeded with {model}\n")
            return result.final_output

        except Exception as e:
            last_error = e
            print(f"⚠️ {agent_name} failed on {model}")
            print(f"Reason: {e}\n")
            await asyncio.sleep(0.5)

    print(f"❌ {agent_name}: all free models failed.")
    raise last_error


In [78]:
print("\n--- 4. Running Final Handoff ---")

final_output = await run_manager_fallback(
    agent_name="Final Manager",
    instructions=final_manager_instructions,
    message="Send a cold sales email to the CEO from Alice",
    tools=drafting_tools,
    handoffs=[
        Agent(
            name="Email Manager",
            instructions=emailer_agent_instructions,
            tools=tools_for_emailer,
            handoff_description="Format and send email"
        )
    ]
)

print("\nFinal output:\n", final_output)



--- 4. Running Final Handoff ---
🔁 Final Manager → Trying model: litellm/openrouter/meta-llama/llama-3.2-3b-instruct:free

Provider List: https://docs.litellm.ai/docs/providers


Provider List: https://docs.litellm.ai/docs/providers



OPENAI_API_KEY is not set, skipping trace export



Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.

⚠️ Final Manager failed on litellm/openrouter/meta-llama/llama-3.2-3b-instruct:free
Reason: litellm.NotFoundError: NotFoundError: OpenrouterException - {"error":{"message":"No endpoints found that support tool use. To learn more about provider routing, visit: https://openrouter.ai/docs/guides/routing/provider-selection","code":404}}

🔁 Final Manager → Trying model: litellm/openrouter/mistralai/mistral-7b-instruct:free

Provider List: https://docs.litellm.ai/docs/providers


Provider List: https://docs.litellm.ai/docs/providers



OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export



Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.

⚠️ Final Manager failed on litellm/openrouter/mistralai/mistral-7b-instruct:free
Reason: litellm.Timeout: Timeout Error: OpenrouterException - litellm.Timeout: Connection timed out. Timeout passed=600.0, time taken=600.782 seconds

🔁 Final Manager → Trying model: litellm/openrouter/tngtech/deepseek-r1t2-chimera:free

Provider List: https://docs.litellm.ai/docs/providers


Provider List: https://docs.litellm.ai/docs/providers


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.

⚠️ Final Manager failed on litellm/openrouter/tngtech/deepseek-r1t2-chimera:free
Reason: litellm.NotFoundError: NotFoundError: OpenrouterException - {"error":{"message":"No endpoints found that support tool use. To learn more about provider routing, visit: https:/

NotFoundError: litellm.NotFoundError: NotFoundError: OpenrouterException - {"error":{"message":"No endpoints found that support tool use. To learn more about provider routing, visit: https://openrouter.ai/docs/guides/routing/provider-selection","code":404}}

OPENAI_API_KEY is not set, skipping trace export


## Conclusion

You have now built a sophisticated multi-agent system that:
1.  Uses parallel processing to generate diversity.
2.  Uses selection logic to pick quality.
3.  Uses specialized tools to interact with real-world APIs (Brevo).
4.  Uses handoffs to delegate specialized tasks (formatting).